In [2]:
from sympy import *
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [5]:
#Symbols
t = symbols('t')
    #states
θ1 = Function('θ1')(t)
θ2 = Function('θ2')(t)
    #coordinates of cart and polemass
x1, y1, x2, y2 = symbols('x1 y1 x2 y2')
    #length link 1, length link 2, COM link 1, mass link 2, mass link 3, I link 1, I link 2, gravity
r1, r2, l, m1, m2, I1, I2, g = symbols('r1 r2 l m1 m2 I1 I2 g')

#Visuals
    #use clean() to render d/dt as dots and remove the (t) in functions
θ1_d_disp = Function('\dot{θ1}')(t)
θ2_d_disp = Function('\dot{θ2}')(t)
θ1_dd_disp = Function('\ddot{θ1}')(t)
θ2_dd_disp = Function('\ddot{θ2}')(t)
dots = {θ1.diff(t):θ1_d_disp, θ2.diff(t):θ2_d_disp, θ1_d_disp.diff(t):θ1_dd_disp, θ2_d_disp.diff(t):θ2_dd_disp}
no_t = { 
    θ1:symbols('θ_1'), θ2:symbols('θ_2'), 
    θ1_d_disp:symbols('\dot{θ_1}'), θ2_d_disp:symbols('\dot{θ_2}'), 
    θ1_dd_disp:symbols('\ddot{θ_1}'), θ2_dd_disp:symbols('\ddot{θ_2}'), 
}
clean = lambda expr : expr.subs(dots).subs(dots).subs(no_t)
    #keep as fraction rather than floating point
half = Rational(1,2)

#Kinetic energy
T1 = half*m1*(l*diff(θ1,t))**2 + half*I1*diff(θ1,t)**2
T2 = half*m2*(r1*diff(θ1,t) + r2*diff(θ2,t))**2 + half*I1*(diff(θ1,t) + diff(θ2,t))**2
T = trigsimp(T1 + T2)

#Potential energy
s1 = sin(θ1)
s12 = sin(θ1 + θ2)
V = m1*g*l*s1 + m2*g*(r1*s1 + r2*s12)

#Lagrangian
L = T - V

clean(L)



I1*\dot{θ_1}**2/2 + I1*(\dot{θ_1} + \dot{θ_2})**2/2 + \dot{θ_1}**2*l**2*m1/2 - g*l*m1*sin(θ_1) - g*m2*(r1*sin(θ_1) + r2*sin(θ_1 + θ_2)) + m2*(\dot{θ_1}*r1 + \dot{θ_2}*r2)**2/2

In [6]:
#Euler-Lagrange
F_θ1, F_θ2 = symbols('F_θ1, F_θ2')
Eq_θ1 = Eq(diff(diff(L, diff(θ1)), t) - diff(L, θ1), F_θ1)
Eq_θ2 = Eq(diff(diff(L, diff(θ2)), t) - diff(L, θ2), F_θ2)

clean(Matrix([
    Eq_θ1.simplify(), 
    Eq_θ2.simplify()
]))

Matrix([
[Eq(F_θ1, I1*\ddot{θ_1} + I1*(\ddot{θ_1} + \ddot{θ_2}) + \ddot{θ_1}*l**2*m1 + g*l*m1*cos(θ_1) + g*m2*(r1*cos(θ_1) + r2*cos(θ_1 + θ_2)) + m2*r1*(\ddot{θ_1}*r1 + \ddot{θ_2}*r2))],
[                                                                       Eq(F_θ2, I1*(\ddot{θ_1} + \ddot{θ_2}) + g*m2*r2*cos(θ_1 + θ_2) + m2*r2*(\ddot{θ_1}*r1 + \ddot{θ_2}*r2))]])

In [4]:
second_derivatives = solve([Eq_x, Eq_θ], [diff(diff(x)), diff(diff(θ))])
x_dd = second_derivatives[diff(diff(x))].simplify()
θ_dd = second_derivatives[diff(diff(θ))].simplify()

In [5]:
clean(Eq(x_dd_disp, x_dd))

Eq(\ddot{x}, (F_x*l + F_θ*sin(θ) + \dot{θ}**2*l**2*m2*cos(θ) - g*l*m2*sin(2*θ)/2)/(l*(m1 + m2*cos(θ)**2)))

In [6]:
clean(Eq(θ_dd_disp, θ_dd))

Eq(\ddot{θ}, (F_x*l*m2*sin(θ) + F_θ*m1 + F_θ*m2 + \dot{θ}**2*l**2*m2**2*sin(2*θ)/2 - g*l*m1*m2*cos(θ) - g*l*m2**2*cos(θ))/(l**2*m2*(m1 + m2*cos(θ)**2)))

In [18]:
x_dd_noFθ = x_dd.subs(Fθ, 0).simplify()
clean(Eq(x_dd_disp, x_dd_noFθ))

Eq(\ddot{x}, (F_x + \dot{θ}**2*l*m2*cos(θ) - g*m2*sin(2*θ)/2)/(m1 + m2*cos(θ)**2))

In [19]:
θ_dd_noFθ = θ_dd.subs(Fθ, 0).simplify()
clean(Eq(θ_dd_disp, θ_dd_noFθ))

Eq(\ddot{θ}, (F_x*sin(θ) + \dot{θ}**2*l*m2*sin(2*θ)/2 - g*m1*cos(θ) - g*m2*cos(θ))/(l*(m1 + m2*cos(θ)**2)))

check work: https://danielpiedrahita.wordpress.com/portfolio/cart-pole-control/
![Alt text](pics/no_inertia.png)

In [25]:
#text format
print('x_dd =', x_dd_noFθ)
print('theta_dd =', θ_dd_noFθ)

x_dd = (F_x - g*m2*sin(2*θ(t))/2 + l*m2*cos(θ(t))*Derivative(θ(t), t)**2)/(m1 + m2*cos(θ(t))**2)
theta_dd = (F_x*sin(θ(t)) - g*m1*cos(θ(t)) - g*m2*cos(θ(t)) + l*m2*sin(2*θ(t))*Derivative(θ(t), t)**2/2)/(l*(m1 + m2*cos(θ(t))**2))
